# Learning from Label Proportions Brain-Computer Interface
The current script contains a basic implementation of LLP for BCI using the data from our original publication [Hübner et al. (2017)](https://arxiv.org/pdf/1701.07213.pdf). You can also take a look at the condensed version from the 2016 NIPS Workshop on _Reliable Machine-Learning in the wild_ [here](https://0586f9b3-a-62cb3a1a-s-sites.googlegroups.com/site/wildml2016nips/KindermansPaper.pdf?attachauth=ANoY7cpKZPUUwVoVdBmJmXGGGbkOtk2rFCN-D0dIPY7MxTG0Fn5DE473nJT-f7pb4TpHVSnkbkiRBurfF82BLE1qwIL4u69Gchko0Bjp5m8fPPorhxQwnDqLrDI3GX9xezDiC30JYprsFiW9yEGcFCeEXxNf3AUvll-bzGSojnp47sZbtCOK51bygl-dMd3RqsSpumd36h4cNKuGMfRXkvt3khtwDwp1F4iLEHXysYvlxOVC8dEGfGA%3D&attredirects=0).

The script performs the following.
* Select a subject
* Set the required variables to fit the experiment
* Load the data (which can be obtained [here](https://zenodo.org/record/192684#.WO6PuBhh3EZ))
* Extract the required variables from the mat files





In [1]:
import numpy as np
from scipy.io import loadmat
from tools.preprocess import flatten_normalise_bias
from sklearn.metrics import roc_auc_score

### Select the subject
options are: 'S1',....,'S13'

In [17]:
subject = 'S5'
data_folder = 'storage/raw_data/'

### Configuration settings
Do not modify these
* **n_trials**: the number of trials/symbols spelled
* **ratio**: the ratio of target to non-target in the different sequence types. This makes it weakly supervised but does not provide explicit label information.
* **pos_neg_ratio**: the global target to non-target ratio. This has makes it weakly supervised. But it is not explicit label information.
* **n_short**: the number of stimuli highlighted in the short sequence. This allows us to assign the stimuli to the short or the long sequence.
* **bad_stimuli**: the stimuli that correspond to the non-selectable symbols.
* **good_stimuli**: the selectable stimuli

In [18]:
n_trials = 63*3
ratio = np.array([2./18.,3./8.,]) # long, short

# the ratio below will be used to estimate X.T y
pos_neg_ratio = np.array([[16./68., -52./68]])
n_short = 12 # Number of stimuli highlighted in a short sequence.
bad_stimuli = [2,4,7,13,17,22,27,32,35,37]
good_stimuli = [i for i in range(42) if i not in bad_stimuli]

### Load the data and extract the components

In [19]:
mf = loadmat('%s/%s'%(data_folder,subject))
channels = [c[0] for c in mf['epo']['clab'][0][0][0]]

# Extract the labels, this is the only form of explicit label information
# It is not used to train the classifier.
labels = mf['epo']['y'][0][0][0].reshape(n_trials,-1)

# Extract the stimuli
# this involves retaining only the good stimuli and re-labelling the other ones
stimuli = mf['epo']['stimuli'][0][0].T[:,good_stimuli]
stimuli = np.dot(stimuli,np.diag(np.r_[:stimuli.shape[1]]+1))-1
stimuli = stimuli.reshape((n_trials,-1,stimuli.shape[1]))

# Extract the groups
group = np.sum(stimuli>=0,axis=2)==n_short

# Extract the eeg
eeg = mf['epo']['x'][0][0]
eeg = eeg.transpose(2,1,0)
eeg = eeg.reshape(n_trials,-1,eeg.shape[1],eeg.shape[2])

### From the components, collect data X and groups G.
The groups G and data X will be used to train the classifier.
Knowing the groups G enables us to estimate the class means without labelled data. These mean estimate will in turn allow us to train a least squares classifier.

In [20]:
X = np.vstack([flatten_normalise_bias(e) for e in eeg])
G = group.flatten()

### Train the classifier
Normally, a least squares classifier has the following solution:
$$ w = (X^T X + \lambda I)^{-1} X^T y.$$

The first component 
$$ (X^T X + \lambda I)^{-1} $$
does not depend on label information and is estimated directly.
The second component
$$ X^T y$$
can be re-written as
$$ \sum_{i=1}^N x_i^T y_i. $$
With $y_i\in\{-1,1\}$ this can be re-written as:
$$N*( p_+ \mu_+  - (1-p_+) \mu_-).$$
Here $N$ is the number of stimuli presented to the user and $p_+$ is the percentage of target responses in X and it is a known parameter of the BCI paragigm.
Therefore we only have to estimate $\mu_+,\mu_-$ to estimate the weight vector $w$.


#### Using LLP to estimate the means $\mu_+,\mu_-$
Now we use the key idea behind LLP to estimate $\mu_+,\mu_-$.

If we know which stimuli $x_i$ belong to the long sequence $l$ with a $\frac{2}{18}$ target to non-target ratio and which stimuli belong to the short sequence $s$ with a $\frac{3}{8}$ target to target ratio we can compute the sequence means $$\mu_l,~~~\mu_s^T.$$

Now using the ratios $\Pi$:
$$\Pi=\left[\begin{array}{cc}
\frac{2}{18} & \frac{3}{8}\\
\frac{16}{18} & \frac{5}{8}
\end{array}\right]
$$
we can estimate the class means.
These means are:
$$
\left[\begin{array}{c}
\mu_+\\
\mu_-
\end{array}\right]
=
(\Pi^T)^{-1}
\left[\begin{array}{c}
\mu_l\\
\mu_s
\end{array}\right]
$$

In [21]:
# Compute the groupwise mean
MU_G = np.array([X[G==g].mean(axis=0) for g in range(2)])

# Estimate the classwise mean from the groupwise mean
P = np.vstack([ratio,1.-ratio])
MU_C = np.dot(np.linalg.inv(P.T),MU_G)



#### Computing $X^T y = N*( p_+ \mu_+  - (1-p_+) \mu_-).$
Using the estimated means we do not require labels for this.

In [22]:
XTy = X.shape[0]*np.dot(pos_neg_ratio,MU_C).T

#### Computing $ (X^T X + \lambda I)^{-1} $
This does not require labels

In [23]:
# Regularisation constant. From ERP experience a value of 10^3 is not absurdly high
lmbd = 1000.
# Compute the regularised design matrix: (X^T X + \lambda I)^{-1}
XTX = np.dot(X.T,X)+lmbd*np.eye(X.shape[1])


#### Compute $ w=(X^T X + \lambda I)^{-1} X^T y$ 

In [24]:
w = np.dot(np.linalg.inv(XTX),XTy)

### Evaluate the classifier (AUC)
and realise that this classifier is trained without labels!

In [25]:
print roc_auc_score(labels.flatten(),np.dot(X,w))

0.867230148232
